# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [19]:
using Manifolds
using Manopt
using LinearAlgebra
using Random

In [20]:
# initialize SPD power manifold P(d)^d1
d = 3  # size of the SPD matrices, i.e., ∈ R^d×d
M_base = SymmetricPositiveDefinite(d)
d1 = 50  # size of the signal
M = PowerManifold(M_base, NestedPowerRepresentation(), d1)

PowerManifold(SymmetricPositiveDefinite(3), NestedPowerRepresentation(), 50)

### Generate a data set

In [31]:
e = 1. * Matrix(I, d, d)
# compute basis on M_base
Θ = get_basis(M_base, e, DefaultOrthonormalBasis())
#  construct data
Q = fill(e, d1)
# draw random tvectors 
τ = 2.  # variance
σ = 0.3  # variance
Xₑⁱ = zeros(manifold_dimension(M_base))
Xₑⁱ[1] = 1.
Xₑ = get_vector(M_base, e, Xₑⁱ, Θ)

Random.seed!(31)
predata = [exp(M_base, e, sqrt(τ) * randn(1)[1] * Xₑ) for i in 1:d1]

data = [exp(M_base, predata[i], random_tangent(M_base, predata[i], Val(:Gaussian), σ)) for i in 1:d1] # ∈ P(3)^d1
log_Q_data = log(M, Q, data)  # ∈ T_e P(3)^d1

50-element Vector{Matrix{Float64}}:
 [-0.8953950713949036 -0.04921489172859089 0.20181985213713866; -0.049214891728590886 -0.011793791491427633 0.15856682497783003; 0.20181985213713868 0.15856682497783003 -0.06877978855094313]
 [-1.4798244335992816 -0.2110952213081288 -0.07272618458656413; -0.2110952213081288 0.4953314080494676 0.1879487936317168; -0.07272618458656413 0.1879487936317168 0.42961945257977197]
 [1.357815947760321 -0.06667536820302258 0.9431237653533775; -0.06667536820302258 0.13699748140258547 -0.07423643907089282; 0.9431237653533776 -0.07423643907089282 -0.14681771109899544]
 [0.40097356322574584 -0.32649969312583005 -0.1278540510876374; -0.32649969312583005 -0.5642078228330145 0.00625417545519899; -0.1278540510876374 0.006254175455199001 -0.3244059728248153]
 [0.1082948879053067 -0.007984499096066827 -0.3217550370912014; -0.007984499096066827 0.03625446058521628 -0.08441942326291604; -0.3217550370912014 -0.08441942326291604 0.2790179034349863]
 [-0.03269669320318254 0.1

We want to investigate how reliable both ranks are, i.e., we don't want that something looks low-rank, but in reality is higher rank

In [28]:
# compute SVD
# compute Gramm Matrix
Gramm_Q = [inner(M_base, e, log_Q_data[i], log_Q_data[j]) for i=1:d1, j=1:d1]
# compute Σ and V
(evals, evecs) = eigen(Symmetric(Gramm_Q), d1-min(d1-1,manifold_dimension(M_base)-1):d1)
# compute low rank approximation of tangent vector
# expoentiate back
# compute error

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
6-element Vector{Float64}:
   2.0437651612108088
   3.509817812831961
   4.4992543949886645
   7.357095071866398
   9.771895218960223
 487.0355318351839
vectors:
50×6 Matrix{Float64}:
 -0.00139289  -0.128407     0.0120486   -0.0158548  -0.0985458  -0.0402217
 -0.0296015   -0.214491    -0.291958    -0.0113016   0.0619462  -0.067285
  0.095434    -0.143854     0.0434354   -0.351617   -0.281746    0.0624237
 -0.0121463    0.227164    -0.0142508    0.197695   -0.142176    0.0182897
 -0.0974963    0.0979468   -0.0970197    0.0576903   0.136954    0.0043241
 -0.170321    -0.0141013    0.171771     0.130915   -0.0288675  -0.00139887
  0.103452    -0.441894     0.174123     0.247269   -0.171582   -0.0859542
 -0.287556     0.0718191    0.108772    -0.0149649  -0.0419965   0.00314198
  0.0959735    0.134651    -0.113368    -0.105045    0.0656367  -0.0603653
 -0.0124193    0.0280613   -0.107587     0.234269    0.0415421   0.0434441

In [32]:
# compute SVD
# B = get_basis(M_base, Q[1], DiagonalizingOrthonormalBasis(log(M_base,Q[1],data[1])))
B = get_basis(M_base, Q[1], DiagonalizingOrthonormalBasis(log_Q_data[1])) # scales with length of log squared -> so we diagonalize with non-normalized geodesics
# B = get_basis(M, Q, DiagonalizingOrthonormalBasis(log_Q_data)) # doesn't compute for powermanifolds -> we should better do it element-wise
B.data.eigenvalues

# we want to compute the metric tensor entries in default ONB at e and then compute g_ij so that we don't have to reevaluate this all the time

# compute rescaled Gramm Matrix
# compute Σ and V
# compute low rank approximation of tangent vector
# expoentiate back
# compute error

6-element Vector{Float64}:
 -0.0
 -0.15738401947442882
 -0.29140548019352847
 -0.0
 -0.020478557618919438
 -0.0